In [10]:
from pynq import Overlay
from pynq import MMIO

# Program bitstream to FPGA
ol = Overlay('./fpga_vae.bit')

# Access to memory map of the AXI GPIO
ADDR_BASE = 0x43C00000
ADDR_RANGE = 0x4000
ol.top_vae_ip_0.mmio = MMIO(ADDR_BASE, ADDR_RANGE)

In [18]:
ol.top_vae_ip_0.write(80,0x1)
ol.top_vae_ip_0.read(80)

1

In [25]:
def float_to_custom_binary(float_value):
    # Determine the sign bit
    sign_bit = 0 if float_value >= 0 else 1

    # Get the absolute value of the float for further processing
    abs_value = abs(float_value)

    # Separate the integer and fractional parts
    integer_part = int(abs_value)
    fractional_part = abs_value - integer_part

    # Convert the integer part to binary (4 bits)
    integer_binary = format(integer_part & 0b1111, '04b')

    # Convert the fractional part to binary (27 bits)
    fractional_binary = ''
    for _ in range(27):
        fractional_part *= 2
        bit = int(fractional_part)
        fractional_binary += str(bit)
        fractional_part -= bit

    # Combine all parts into a single binary string
    custom_binary = f"{sign_bit}{integer_binary}{fractional_binary}"
    binary_value = int(custom_binary, 2)
    binary_representation = bin(binary_value)

    return binary_value

# Input from the user
user_input = input("Please enter a float value: ")

try:
    float_value = float(user_input)
    custom_binary = float_to_custom_binary(float_value)
    print("Custom binary representation:", custom_binary, type(custom_binary))
#     print("Custom binary representation:", int(custom_binary,2), type(int(custom_binary,2)))
# #     binary_representation = bin(binary_value)
#     print(f"Representasi biner dengan prefix: {bin(int(custom_binary,2))}")
except ValueError:
    print("Invalid input. Please enter a valid float value.")

# sigmoid_opt_x = [-4.09263072, -1.81172374 , 1.81171603 , 4.09261913]
# for s in sigmoid_opt_x:
#     print(float_to_custom_binary(s))

Please enter a float value: 10
Custom binary representation: 1342177280 <class 'int'>


In [40]:
def pad_binary_to_32bit(binary_str):
    """Pad binary string with leading zeros to make it 32 bits."""
    padded_binary = binary_str.zfill(32)  # Menambahkan leading zeros hingga panjangnya 32
    return padded_binary

def binary_to_float(binary_str):
    """Convert a 32-bit binary string to a float."""
    sign = int(binary_str[0])
    integer_part = int(binary_str[1:5], 2)
    fractional_part = binary_str[5:]
    frac_value = 0.0
    for i, bit in enumerate(fractional_part):
        if bit == '1':
            frac_value += 1 / (2 ** (i + 1))
    float_value = integer_part + frac_value
    if sign == 1:
        float_value = -float_value
    return float_value

# Input from the user
user_input = input("Please enter a binary value: ")

float_data = binary_to_float(user_input)
print(f"Custom binary representation: {float_data:.10f}")

Please enter a binary value: 01
Custom binary representation: 1.0000000000


In [43]:
# input array
ol.top_vae_ip_0.write(0,float_to_custom_binary(1))
ol.top_vae_ip_0.write(4,float_to_custom_binary(1))
ol.top_vae_ip_0.write(8,float_to_custom_binary(1))
ol.top_vae_ip_0.write(12,float_to_custom_binary(1))
ol.top_vae_ip_0.write(16,float_to_custom_binary(0))
ol.top_vae_ip_0.write(20,float_to_custom_binary(1))
ol.top_vae_ip_0.write(24,float_to_custom_binary(1))
ol.top_vae_ip_0.write(28,float_to_custom_binary(1))
ol.top_vae_ip_0.write(32,float_to_custom_binary(1))

In [44]:
# Output
print(ol.top_vae_ip_0.read(36))  # Membaca dan mencetak alamat 36
print(ol.top_vae_ip_0.read(40))  # Membaca dan mencetak alamat 40
print(ol.top_vae_ip_0.read(44))  # Membaca dan mencetak alamat 44
print(ol.top_vae_ip_0.read(48))  # Membaca dan mencetak alamat 48
print(ol.top_vae_ip_0.read(52))  # Membaca dan mencetak alamat 52
print(ol.top_vae_ip_0.read(56))  # Membaca dan mencetak alamat 56
print(ol.top_vae_ip_0.read(60))  # Membaca dan mencetak alamat 60
print(ol.top_vae_ip_0.read(64))  # Membaca dan mencetak alamat 64
print(ol.top_vae_ip_0.read(68))  # Membaca dan mencetak alamat 68

35182933
5298105
29963945
1297296
136029387
1140632
45412147
4340476
43711578


In [45]:
addresses = [36, 40, 44, 48, 52, 56, 60, 64, 68]
for addr in addresses:
    print(binary_to_float(pad_binary_to_32bit(bin(ol.top_vae_ip_0.read(addr))[2:])))

0.26213327795267105
0.03947395831346512
0.22324878722429276
0.009665608406066895
1.0134979113936424
0.00849837064743042
0.33834686130285263
0.03233906626701355
0.3256766349077225


In [39]:
for addr in addresses:
    print(bin(ol.top_vae_ip_0.read(addr))[2:])

111010000011000000100010101
10000000001000010000001000101111
111001000011000110111101101
10000000001010100100100011101100
111111110101011000000010110
10000000001011000101001111010010
111011011101110011010000101
10000000001000101000011010100000
111011001110101011111111110
